In [ ]:
import pandas as pd
import re 

def clean_text(text):
    # remove multiple newlines
    text = re.sub(r'\n+', '\n', text)
    # remove multiple spaces
    text = re.sub(r' +', ' ', text)
    # strip leading and trailing whitespace
    text = text.strip()
    # remove triple backticks and the word markdown
    text = text.replace("```", "").replace("markdown", "")
    # remove first and last newlines
    text = text.lstrip('\n').rstrip('\n')
    return text


df = pd.read_csv("data/processed/resumes.csv")
df.drop(columns=['resume', 'anonymized', 'reformatted'], inplace=True)
df.rename(columns={'localized': 'resume'}, inplace=True)
df['resume'] = df['resume'].apply(clean_text)
df.head(5)

In [ ]:
from IPython.display import display, Markdown
from transformers import pipeline
import re
from collections import Counter

# 1. Load a small local NER model
ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)


# 3. Simple helper: check if mention is in educational context
def is_education_context(sentence, org):
    keywords = ["study", "studied", "graduated", "degree", "phd", "bachelor", 
                "master", "diploma", "alumni", "completed", "course", "education"]
    window = 10  # number of words around the org to scan
    words = sentence.lower().split()
    org_words = org.lower().split()
    try:
        idx = words.index(org_words[0])
    except ValueError:
        return False
    context = words[max(0, idx-window): idx+len(org_words)+window]
    return any(kw in context for kw in keywords)

# 4. Process each sentence
counts = Counter()
    

for i, row in df.iterrows():
    if 'education' in row['resume'].lower():
        # print the sentences with the word education
        sentences = re.split(r'(?<=[.!?]) +', row['resume'])
        for sent in sentences:
            if 'education' in sent.lower():
                orgs = [ent['word'] for ent in ner(sent) if ent['entity_group'] == "ORG"]
                edu_orgs = [org for org in orgs if is_education_context(sent, org)]
                counts.update(edu_orgs)

print(counts)
print(sum(counts.values()))

In [ ]:
from IPython.display import display, Markdown

# print all resumes
for i, row in df.iterrows():
    display(Markdown(f"### Resume {i+1}\n{row['resume']}\n"))
    break
    

In [ ]:
identical = {
    "NUS": ["National University of Singapore", "National University of Singapore Singapore", "NUS Singapore", "NUS University", "NUS University Singapore"],
    "NTU": ["Nanyang Technological University", "Nanyang Technological University Singapore", "NTU Singapore", "NTU University", "NTU University Singapore"],
    "SMU": ["Singapore Management University", "Singapore Management University Singapore", "SMU Singapore", "SMU University", "SMU University Singapore"],
    "SUTD": ["Singapore University of Technology and Design", "Singapore University of Technology and Design Singapore", "SUTD Singapore", "SUTD University", "SUTD University Singapore"],
    "SIT": ["Singapore Institute of Technology", "Singapore Institute of Technology Singapore", "SIT Singapore", "SIT University", "SIT University Singapore"],
    "SUS": ["Singapore University of Social Sciences", "Singapore University of Social Sciences Singapore", "SUSS Singapore", "SUSS University", "SUSS University Singapore"],
    "INSEAD": ["INSEAD Business School", "INSEAD Business School Singapore", "INSEAD Singapore", "INSEAD University", "INSEAD University Singapore"],
    "ESSEC": ["ESSEC Business School", "ESSEC Business School Singapore", "ESSEC Singapore", "ESSEC University", "ESSEC University Singapore"],
    "SP": ["Singapore Polytechnic", "Singapore Polytechnic Singapore", "SP Singapore", "SP University", "SP University Singapore"],
    "NP": ["Ngee Ann Polytechnic", "Ngee Ann Polytechnic Singapore", "NP Singapore", "NP University", "NP University Singapore"],
    "TP": ["Temasek Polytechnic", "Temasek Polytechnic Singapore", "TP Singapore", "TP University", "TP University Singapore"],
    "RP": ["Republic Polytechnic", "Republic Polytechnic Singapore", "RP Singapore", "RP University", "RP University Singapore"],
    "NYP": ["Nanyang Polytechnic", "Nanyang Polytechnic Singapore", "NYP Singapore", "NYP University", "NYP University Singapore"],
    "RI": ["Raffles Institution", "Raffles Institution Singapore", "RI Singapore", "RI School", "RI School Singapore"],
    "RJC": ["Raffles Junior College", "Raffles Junior College Singapore", "RJC Singapore", "RJC School", "RJC School Singapore"],
    "HCI": ["Hwa Chong Institution", "Hwa Chong Institution Singapore", "HCI Singapore", "HCI School", "HCI School Singapore"],
    "ACJC": ["Anglo-Chinese Junior College", "Anglo-Chinese Junior College Singapore", "ACJC Singapore", "ACJC School", "ACJC School Singapore"],
    "ACSI": ["Anglo-Chinese School (Independent)", "Anglo-Chinese School (Independent) Singapore", "ACSI Singapore", "ACSI School", "ACSI School Singapore"],
    "DHS": ["Dunman High School", "Dunman High School Singapore", "DHS Singapore", "DHS School", "DHS School Singapore"],
    "MGS": ["Methodist Girls' School", "Methodist Girls' School Singapore", "MGS Singapore", "MGS School", "MGS School Singapore"]

}
counts = Counter()
# find all counts of identical keys in each resume
for i, row in df.iterrows():
    for key in identical.keys():
        for variant in identical[key]:
            counts[key] += row['resume'].count(variant)
        counts[key] += row['resume'].count(key)
print(counts)

In [ ]:
import matplotlib.pyplot as plt

# Only p]lot the educational institutions in identical.keys()
plt.figure(figsize=(10, 6))
plt.barh([key for key in counts.keys() if key in identical], [counts[key] for key in counts.keys() if key in identical], color='skyblue')
plt.xlabel('Number of Mentions')
plt.title('Educational Institutions Mentioned in Resumes')
plt.tight_layout()
plt.show()

In [ ]:
aggregated_data = {
 'Singapore Manufacturing Company': 2,
 'Oracle': 2,
 'Singtel': 2,
 'DBS Bank': 17,
 'Frost Design': 1,
 'Singapore Armed Forces': 4,
 'Ministry of Health': 1,
 'Cyber Security Agency of Singapore': 2,
 'National Cyber Response Team': 1,
 'Public Service Division': 1,
 'ST Engineering': 2,
 'OCBC Bank': 8,
 'Raffles Institution': 1,
 'Little Skool-House International': 1,
 'McDonald’s Singapore': 1,
 'The Learning Lab': 2,
 'Healthway Medical Group': 2,
 'Home Nursing Foundation': 1,
 'SATS Ltd': 2,
 'Charles & Keith': 1,
 'Metro Holdings': 1,
 'H&M': 1,
 'Takashimaya': 1,
 'Raffles Medical Group': 3,
 'TCCC (The Coca-Cola Company)': 1,
 'National University of Singapore': 5,
 'Singapore Dental College': 1,
 'Fitness First': 1,
 'National Dental Centre Singapore': 1,
 'Singapore Red Cross': 2,
 'Ministry of Sustainability and the Environment': 1,
 'Singapore Polytechnic': 1,
 'National Parks Board': 1,
 'StarHub': 1,
 'Singapore Airlines': 1,
 'NTUC FairPrice': 1,
 'Chan Brothers Travel': 1,
 'Singapore Institute of Management': 1,
 'Temasek Polytechnic': 1,
 'Singapore Air Force': 2,
 'AXA Insurance': 1,
 'GWI Help Desk Solutions': 1,
 'SingHealth': 1,
 'HealthMetrics Pte Ltd': 1,
 'Mediacorp': 1,
 'Singapore Management University': 1,
 'Changi Airport Group': 1,
 'CapitaLand': 1,
 'TIBCO': 1,
 'NTUC': 1,
 'Time Out': 1,
 'The Honeycombers': 1,
 'CNA': 2,
 'Deloitte': 1,
 'Club 21': 2,
 'MediaTek': 1,
 'Nanyang Technological University': 1,
 'City Harvest Church': 1,
 'KRONOS': 1,
 'JD Edwards': 1,
 'Tech Solutions Pte Ltd': 2,
 'Health Promotion Board': 1,
 'OCBC Bank': 2,
 'UOB': 2,
 'United Overseas Bank': 1,
 'JP Morgan': 1,
 'Bloomberg': 1,
 'Singapore International School': 1,
 'Industrial Welding Company': 1,
 'Ministry of Transport': 2,
 'Singapore Aircraft Corporation': 1,
 'Singapore Aviation Services': 1
}

In [ ]:
# plot the top 20 companies

top_companies = dict(sorted(aggregated_data.items(), key=lambda item: item[1], reverse=True)[:25])
plt.barh(list(top_companies.keys()), list(top_companies.values()))
plt.xlabel('Number of Jobs')
plt.title('Top 25 Companies In Resumes by Job Count')
plt.show()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

res = []
for resume in df['resume']:
    prompt = f"""Your task is to generate a realistic name for the following anonymized resume from Singapore.
You will see a [Candidate Name] placeholder. Your job is to suggest a realistic name that fits the profile. Output the full name of the person only.
{resume}
Only generate the person's full name."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=32,
        do_sample=True,
        temperature=0.7
    )
    output = tokenizer.decode(generated_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    res.append(output.strip())
print("generate_text:", res)


In [ ]:
res

In [ ]:
from transformers import pipeline

# For prototyping, we can use a T5 model; in practice, you'd fine-tune it
name_predictor = pipeline(
    task="text2text-generation",
    model="t5-base",   # replace with your fine-tuned model
    tokenizer="t5-base"
)

resume_text = df['resume'][0]

# Prompt format – important for training consistency
prompt = f"Given the following anonymized resume from Singapore, generate an appropriate name of the person:\n{resume_text}\n\n Name - "

result = name_predictor(prompt)
# Only print the generated name
print(result[0]['generated_text'])

In [ ]:
print(result)